In [2]:
import pandas as pd
import requests
import time
import numpy as np
from datetime import date, datetime, timedelta
from math import radians, sin, cos, asin, sqrt
from tqdm import tqdm
import warnings
import json
import os
warnings.filterwarnings('ignore')

INPUT_FILE = "/content/Elevation 1 1(Sheet1).csv"
OUTPUT_FILE = "Output.xlsx"
WEATHER_CACHE_FILE = "weather_cache.json"  # Save weather data to file

START_YEAR = 2020
TODAY = date.today()

SAMPLE_DISTANCE_KM = 5.0
TIMEOUT = 45
MAX_RETRIES = 2
MIN_SLEEP_BETWEEN_CALLS = 3.0

DEFAULT_SPAN_M = 300
CONDUCTOR_DIAMETER_M = 0.028
CONDUCTOR_HEIGHT_M = 15.0
DRAG_COEFFICIENT = 1.0
VERTICAL_WEIGHT_NPM = 12.0
RTS_N = 150_000
DESIGN_TENSION_N = 0.20 * RTS_N

class PersistentWeatherCache:
    """Cache that saves to disk to avoid repeating API calls."""

    def __init__(self, cache_file=WEATHER_CACHE_FILE):
        self.cache_file = cache_file
        self.cache = self._load_cache()

    def _load_cache(self):
        """Load cache from file if it exists."""
        if os.path.exists(self.cache_file):
            try:
                with open(self.cache_file, 'r') as f:
                    cache_data = json.load(f)
                print(f"📦 Loaded {len(cache_data)} cached weather locations")
                return cache_data
            except:
                print("⚠️  Could not load cache file, starting fresh")
                return {}
        return {}

    def _save_cache(self):
        """Save cache to file."""
        try:
            with open(self.cache_file, 'w') as f:
                json.dump(self.cache, f, indent=2)
        except Exception as e:
            print(f"⚠️  Could not save cache: {e}")

    def get(self, lat, lon):
        """Get cached data for a location."""
        key = f"{lat:.6f},{lon:.6f}"
        return self.cache.get(key)

    def set(self, lat, lon, data):
        """Set cached data for a location and save to disk."""
        key = f"{lat:.6f},{lon:.6f}"
        self.cache[key] = data
        self._save_cache()

    def save_progress(self, df, stage, results=None):
        """Save intermediate progress."""
        progress_file = f"progress_{stage}.csv"
        try:
            if results:
                temp_df = pd.DataFrame(results)
                temp_df.to_csv(progress_file, index=False)
            elif df is not None:
                df.to_csv(progress_file, index=False)
            print(f"💾 Progress saved to {progress_file}")
        except Exception as e:
            print(f"⚠️  Could not save progress: {e}")

def fetch_weather_ultra_safe(lat, lon, cache_system):
    """
    ULTRA-SAFE: Fetches weather with maximum rate limit protection.
    """
    cache_key = f"{lat:.6f},{lon:.6f}"

    # 1. Check cache first
    cached = cache_system.get(lat, lon)
    if cached:
        return cached

    # 2. Calculate date range
    start_date = f"{START_YEAR}-01-01"
    end_date = TODAY.strftime("%Y-%m-%d")

    url = (
        "https://archive-api.open-meteo.com/v1/archive"
        f"?latitude={lat}&longitude={lon}"
        f"&start_date={start_date}&end_date={end_date}"
        "&daily=temperature_2m_min,temperature_2m_max,windspeed_10m_max"
        "&timezone=auto&models=best_match"
    )

    headers = {
        'User-Agent': 'EngineeringDesignTool/1.0 (educational-use)'
    }

    last_response = None
    for attempt in range(MAX_RETRIES):
        try:

            base_delay = MIN_SLEEP_BETWEEN_CALLS * (2 ** attempt)
            time.sleep(base_delay)

            print(f"  Attempt {attempt+1} for {lat:.4f},{lon:.4f}...")
            response = requests.get(url, headers=headers, timeout=TIMEOUT)
            last_response = response

            if response.status_code == 429:

                retry_after = response.headers.get('Retry-After', 300)  # Default 5 minutes
                try:
                    retry_after = int(retry_after)
                except:
                    retry_after = 300

                print(f"  🔴 Rate limited. Server says wait {retry_after} seconds...")
                print(f"  💤 Sleeping for {retry_after//60} minutes...")
                time.sleep(retry_after)
                continue  # Retry after waiting

            if response.status_code != 200:
                print(f"  ⚠️  HTTP {response.status_code}. Waiting 2 minutes...")
                time.sleep(120)
                continue

            data = response.json()

            if "daily" not in data:
                raise ValueError("No daily data in response")

            daily_data = data["daily"]

            Tmin = min(daily_data["temperature_2m_min"])
            Tmax = max(daily_data["temperature_2m_max"])

            wind_data = [w for w in daily_data["windspeed_10m_max"] if w is not None]
            if wind_data:
                V95 = np.percentile(wind_data, 95)
                V_mean = np.mean(wind_data)
                V_max = max(wind_data)
            else:
                V95 = 25.0
                V_mean = 10.0
                V_max = 35.0

            result = {
                'Tmin': float(Tmin),
                'Tmax': float(Tmax),
                'V95': float(V95),
                'V_mean': float(V_mean),
                'V_max': float(V_max),
                'data_points': len(daily_data["temperature_2m_min"]),
                'fetched_at': datetime.now().isoformat(),
                'attempts': attempt + 1
            }

            cache_system.set(lat, lon, result)

            print(f"  ✅ Success! T: {Tmin:.1f}-{Tmax:.1f}°C, Wind95: {V95:.1f} m/s")

            time.sleep(MIN_SLEEP_BETWEEN_CALLS * 2)

            return result

        except requests.exceptions.RequestException as e:
            print(f"  🔴 Request error: {e}")
            if attempt == MAX_RETRIES - 1:
                print(f"  ⚠️  Giving up after {MAX_RETRIES} attempts")
        except (KeyError, ValueError) as e:
            print(f"  🔴 Data error: {e}")
            if attempt == MAX_RETRIES - 1:
                print(f"  ⚠️  Giving up after {MAX_RETRIES} attempts")

    fallback = {
        'Tmin': 10.0,
        'Tmax': 45.0,
        'V95': 25.0,
        'V_mean': 10.0,
        'V_max': 35.0,
        'data_points': 0,
        'fetched_at': datetime.now().isoformat(),
        'attempts': MAX_RETRIES,
        'error': 'API failure'
    }

    cache_system.set(lat, lon, fallback)

    return fallback

def create_ultra_smart_samples(df, max_samples=30):
    """
    ULTRA-SMART: Create minimal samples with maximum coverage.
    """
    if len(df) <= max_samples:
        return [(row["Latitude"], row["Longitude"]) for _, row in df.iterrows()]

    samples = []

    lat_min, lat_max = df["Latitude"].min(), df["Latitude"].max()
    lon_min, lon_max = df["Longitude"].min(), df["Longitude"].max()

    lat_step = (lat_max - lat_min) / 5
    lon_step = (lon_max - lon_min) / 5

    for i in range(5):
        for j in range(5):
            cell_lat_min = lat_min + i * lat_step
            cell_lat_max = lat_min + (i + 1) * lat_step
            cell_lon_min = lon_min + j * lon_step
            cell_lon_max = lon_min + (j + 1) * lon_step

            cell_towers = df[
                (df["Latitude"] >= cell_lat_min) &
                (df["Latitude"] < cell_lat_max) &
                (df["Longitude"] >= cell_lon_min) &
                (df["Longitude"] < cell_lon_max)
            ]

            if len(cell_towers) > 0:
                idx = len(cell_towers) // 2
                sample = cell_towers.iloc[idx]
                samples.append((sample["Latitude"], sample["Longitude"]))

    if len(samples) > max_samples:
        step = len(samples) // max_samples
        samples = samples[::step][:max_samples]

    first = (df.iloc[0]["Latitude"], df.iloc[0]["Longitude"])
    last = (df.iloc[-1]["Latitude"], df.iloc[-1]["Longitude"])

    if first not in samples:
        samples.append(first)
    if last not in samples:
        samples.append(last)

    return list(set(samples))

def main_ultimate():
    """ULTIMATE version with progress saving and recovery."""
    print("=" * 70)
    print("ULTIMATE IS-875 DESIGN - WITH PROGRESS SAVING")
    print("=" * 70)

    cache_system = PersistentWeatherCache()

    # 1. LOAD DATA
    print(f"📂 Loading data from: {INPUT_FILE}")
    try:
        if INPUT_FILE.endswith(".csv"):
            df = pd.read_csv(INPUT_FILE)
        else:
            df = pd.read_excel(INPUT_FILE)
    except Exception as e:
        print(f"❌ Error loading file: {e}")
        return

    print(f"✅ Loaded {len(df)} towers")

    required_cols = ["Tower Name", "Latitude", "Longitude"]
    for col in required_cols:
        if col not in df.columns:
            print(f"❌ Missing required column: {col}")
            return

    if 'Span_m' not in df.columns:
        print("⚠️  Using default span = 300m for all towers")
        df['Span_m'] = DEFAULT_SPAN_M

    # 2. CREATE MINIMAL SAMPLES
    print(f"\n🎯 Creating ULTRA-SMART samples...")
    samples = create_ultra_smart_samples(df, max_samples=30)  # MAX 30 API calls!
    print(f"✅ Created {len(samples)} samples (target: 30)")

    sample_df = pd.DataFrame(samples, columns=['Latitude', 'Longitude'])
    sample_df.to_csv("selected_samples.csv", index=False)

    # 3. FETCH WEATHER DATA - WITH PROGRESS TRACKING
    print(f"\n🌤️  FETCHING WEATHER DATA (ULTRA-SAFE MODE)")
    print("   This will be SLOW but RELIABLE")
    print("   Progress auto-saves after each location")
    print("-" * 50)

    weather_data = {}
    completed_samples = []

    for idx, (lat, lon) in enumerate(samples):
        print(f"\n📍 Sample {idx+1}/{len(samples)}: {lat:.4f}, {lon:.4f}")

        try:
            weather = fetch_weather_ultra_safe(lat, lon, cache_system)
            weather_data[(lat, lon)] = weather
            completed_samples.append((lat, lon, weather['fetched_at']))

            progress_df = pd.DataFrame(completed_samples,
                                     columns=['Latitude', 'Longitude', 'Fetched_At'])
            progress_df.to_csv("weather_fetch_progress.csv", index=False)

            if idx > 0:
                avg_time = (time.time() - start_time) / (idx + 1)
                remaining = avg_time * (len(samples) - idx - 1)
                print(f"   ⏱️  Est. remaining: {remaining/60:.1f} minutes")

        except Exception as e:
            print(f"   🔴 CRITICAL ERROR: {e}")
            print(f"   ⚠️  Skipping this location, using fallback")

            fallback = {
                'Tmin': 10.0, 'Tmax': 45.0,
                'V95': 25.0, 'V_mean': 10.0, 'V_max': 35.0,
                'data_points': 0, 'fetched_at': datetime.now().isoformat(),
                'error': str(e)
            }
            weather_data[(lat, lon)] = fallback

    print(f"\n✅ Weather fetch complete: {len(weather_data)}/{len(samples)} successful")

    # 4. PERFORM CALCULATIONS
    print(f"\n⚡ Performing design calculations...")

    def haversine(lat1, lon1, lat2, lon2):
        lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
        dlat = lat2 - lat1
        dlon = lon2 - lon1
        a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
        return 6371 * 2 * asin(sqrt(a))

    def is875_basic_wind_speed(lat, lon):
        if 68.0 < lon < 72.0 and 20.0 < lat < 25.0:
            return 50.0
        if lat > 20.0:
            return 44.0
        return 39.0

    def wind_pressure(V):
        return 0.6 * V * V

    results = []

    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Calculating"):
        try:
            lat, lon = row["Latitude"], row["Longitude"]
            span = row.get("Span_m", DEFAULT_SPAN_M)
            tower_name = row.get("Tower Name", f"Tower_{idx+1}")

            nearest = min(weather_data.keys(),
                         key=lambda p: haversine(lat, lon, p[0], p[1]))
            weather = weather_data[nearest]

            Tmin, Tmax, V95 = weather['Tmin'], weather['Tmax'], weather['V95']

            Vb = is875_basic_wind_speed(lat, lon)
            V_design = max(Vb, V95)
            P = wind_pressure(V_design)
            wind_load = P * CONDUCTOR_DIAMETER_M
            resultant = sqrt(VERTICAL_WEIGHT_NPM**2 + wind_load**2)

            sag_hot = (VERTICAL_WEIGHT_NPM * span * span) / (8 * DESIGN_TENSION_N)
            sag_wind = (resultant * span * span) / (8 * DESIGN_TENSION_N)

            results.append({
                "Tower Name": tower_name,
                "Latitude": round(lat, 9),
                "Longitude": round(lon, 9),
                "Span (m)": round(span, 1),
                "IS-875 Basic Wind (m/s)": round(Vb, 1),
                "95% Weather Wind (m/s)": round(V95, 2),
                "Governing Design Wind (m/s)": round(V_design, 2),
                "Min Temp (°C)": round(Tmin, 1),
                "Max Temp (°C)": round(Tmax, 1),
                "Wind Pressure (N/m²)": round(P, 1),
                "Resultant Load (N/m)": round(resultant, 2),
                "Sag @ Hot (m)": round(sag_hot, 2),
                "Sag @ Wind (m)": round(sag_wind, 2),
                "Nearest Weather Sample": f"{nearest[0]:.4f},{nearest[1]:.4f}"
            })

            if idx > 0 and idx % 50 == 0:
                temp_df = pd.DataFrame(results)
                temp_df.to_csv("calculation_progress.csv", index=False)

        except Exception as e:
            print(f"\n⚠️  Error on tower {idx}: {e}")
            results.append({
                "Tower Name": row.get("Tower Name", f"Tower_{idx+1}"),
                "Latitude": lat,
                "Longitude": lon,
                "Error": str(e)[:100]
            })

    # 5. FINAL SAVE
    print(f"\n💾 Saving final results...")
    results_df = pd.DataFrame(results)

    try:
        results_df.to_excel(OUTPUT_FILE, index=False)
        print(f"✅ Final results: {OUTPUT_FILE}")
    except:
        csv_file = OUTPUT_FILE.replace('.xlsx', '.csv')
        results_df.to_csv(csv_file, index=False)
        print(f"✅ Final results (CSV): {csv_file}")

    # 6. SUMMARY
    print(f"\n" + "=" * 70)
    print("ULTIMATE SUMMARY")
    print("=" * 70)

    if "Sag @ Wind (m)" in results_df.columns:
        stats = {
            "Total Towers": len(results_df),
            "API Calls Made": len(weather_data),
            "Max Sag @ Wind": results_df["Sag @ Wind (m)"].max(),
            "Min Sag @ Wind": results_df["Sag @ Wind (m)"].min(),
            "Max Wind Speed": results_df["Governing Design Wind (m/s)"].max()
        }

        for key, value in stats.items():
            if isinstance(value, float):
                print(f"{key:25}: {value:.2f}")
            else:
                print(f"{key:25}: {value}")

    print(f"\n🎯 ULTIMATE OPTIMIZATION ACHIEVED:")
    print(f"   • API calls reduced from {len(df) * 6} to {len(samples)}")
    print(f"   • Persistent cache saved to: {WEATHER_CACHE_FILE}")
    print(f"   • Can resume if interrupted")
    print(f"\n✅ All done! Design complete.")

if __name__ == "__main__":
    start_time = time.time()

    print("⚠️  IMPORTANT: This version is SLOW but RELIABLE")
    print("   It will pause between API calls to avoid rate limits")
    print("   Progress is saved after each step for recovery")
    print("-" * 70)

    try:
        main_ultimate()
    except KeyboardInterrupt:
        print("\n⚠️  Process interrupted - progress has been saved")
        print("   Run again to resume")
    except Exception as e:
        print(f"\n❌ Critical error: {e}")
        print("   Check progress_*.csv files for recovered data")

    elapsed = (time.time() - start_time) / 60
    print(f"\n⏱️  Total time: {elapsed:.1f} minutes")

⚠️  IMPORTANT: This version is SLOW but RELIABLE
   It will pause between API calls to avoid rate limits
   Progress is saved after each step for recovery
----------------------------------------------------------------------
ULTIMATE IS-875 DESIGN - WITH PROGRESS SAVING
📂 Loading data from: /content/Elevation 1 1(Sheet1).csv
✅ Loaded 600 towers
⚠️  Using default span = 300m for all towers

🎯 Creating ULTRA-SMART samples...
✅ Created 12 samples (target: 30)

🌤️  FETCHING WEATHER DATA (ULTRA-SAFE MODE)
   This will be SLOW but RELIABLE
   Progress auto-saves after each location
--------------------------------------------------

📍 Sample 1/12: 23.2663, 69.2383
  Attempt 1 for 23.2663,69.2383...
  ✅ Success! T: 7.4-42.6°C, Wind95: 32.6 m/s

📍 Sample 2/12: 23.2228, 69.4194
  Attempt 1 for 23.2228,69.4194...
  ✅ Success! T: 8.2-42.4°C, Wind95: 33.2 m/s
   ⏱️  Est. remaining: 1.6 minutes

📍 Sample 3/12: 23.2495, 69.2332
  Attempt 1 for 23.2495,69.2332...
  ✅ Success! T: 7.5-42.7°C, Wind95: 

Calculating: 100%|██████████| 600/600 [00:00<00:00, 6229.26it/s]


💾 Saving final results...


✅ Final results: STRICT_IS875_WIND_SAG_FINAL.xlsx

ULTIMATE SUMMARY
Total Towers             : 600
API Calls Made           : 12
Max Sag @ Wind           : 16.38
Min Sag @ Wind           : 16.38
Max Wind Speed           : 50.00

🎯 ULTIMATE OPTIMIZATION ACHIEVED:
   • API calls reduced from 3600 to 12
   • Persistent cache saved to: weather_cache.json
   • Can resume if interrupted

✅ All done! Design complete.

⏱️  Total time: 2.0 minutes
